# Multi-Agent Retrieval-Augmented Generation (RAG)

## Agent Graph Framework

In [13]:
class Graph:
    def __init__(self, START):
        self.START = START

    def start(self):
        self.START.run()

In [14]:
class Node:
    def __init__(self, name=None):
        self.name = name

    def run(self):
        pass

In [15]:
class Agent(Node):
    def __init__(self, condition, name=None): 
        super().__init__(name)
        self.condition = condition
        self.yes = None
        self.no = None

    def add_yes(self, node):
        self.yes = node

    def add_no(self, node):
        self.no = node
    
    def run(self):
        print(f"--Agent {self.name} Called--")
        if self.condition():
            self.yes.run()
        else:
            self.no.run()

In [16]:
class Tool(Node):
    def __init__(self, action, name=None): 
        super().__init__(name)
        self.action = action
    
    def run(self):
        print(f"--Tool {self.name} Called--")
        self.action()

## Building Multi-Agent RAG with above Framework

In [17]:
import google.generativeai as genai

genai.configure(api_key="Enter your API Key")

llm = genai.GenerativeModel("gemini-1.5-flash")

In [18]:

# Sample inputs

question = "What is langchain?"
document_text = "LangChain is a framework designed to help developers create applications powered by language models. It provides tools to connect LLMs with external sources of data and create complex workflows."

# question = "What is langchain?"
# document_text = ""

# question = "What is google?"
# document_text = ""

In [19]:
# Check if the document contains enough information to answer the question
def can_answer_question():
    # Define a prompt to check if the question can be answered

    # "Does the document contain enough information to answer the question? " \
    check_prompt = f"Document: {document_text}\n\nQuestion: {question}\n\n" \
                   "Can you answer the above question with or without the given document? " \
                   "Respond with 'yes' or 'no'."

    response = llm.generate_content(check_prompt).text
    print(response)
    if "yes" in response.lower():
        return True # document contains sufficient information
    else:
        return False # document lacks sufficient information

In [20]:
# Generate an answer if the document is sufficient
def generate_answer():
    # Define a prompt to generate the answer based on the document
    answer_prompt = f"Document: {document_text}\n\nQuestion: {question}\n\nAnswer:"
    answer = llm.generate_content(answer_prompt)
    print(answer.text)

In [21]:
def search_google():
    #  Search Google for the answer
    print("Searching Google for the answer...")

In [22]:
agent1 = Agent(condition=lambda: can_answer_question(), name = "Agent1")
generate = Tool(action=lambda: generate_answer(), name = "Generate")
search = Tool(action=lambda: search_google(), name="Google Search")

agent1.add_yes(generate)
agent1.add_no(search)

In [23]:
graph = Graph(START=agent1)

In [24]:
graph.start()

--Agent Agent1 Called--
yes 

--Tool Generate Called--
LangChain is a framework that helps developers build applications powered by language models. It connects LLMs with external data sources and allows for the creation of complex workflows. 

